# CUDA GPU 并行编程 

## 目录

1. [Hello World](#Hello-World) :第一个Hello World演示程序
2. [Add](#Add) :利用GPU实现两个整数的相加。

## Hello World
---

**路径**: study_notes/cuda/demos/1/kernel.cu

```c++
# include <iostream>

__global__ void kernel(void) {
}

int main(void)
{
        kernel<<<1,1>>>();
        std::cout << "hello world" << std::endl;
        return 0;
}
```

**程序分析：**
- `__global__`: 告诉编译函数应该在GPU设备上面运行；
- 未加修饰符的函数将在CPU主机上面运行；
- `<<<1,1>>>`: 主机中调用GPU设备上运行的函数的标识符，里面的数字分别代表在GPU设备上面开辟的线程块的数目和每个线程块中的线程数目。

## Add

---

**路径**: study_notes/cuda/demos/2/kernel.cu

```c++
# include <iostream>
# include <cstdlib>

using namespace std;

__global__ void add(int a, int b, int *c) {
        *c = a + b;
}

int main(int argc, const char * const * argv)
{
        int a, b, c, *dev_c;

        if (argc != 3) {
                cout << "please input two numbers a and b." << endl;
                return 1;
        }

        a = atoi(argv[1]);
        b = atoi(argv[2]);

        cudaMalloc((void**)&dev_c, sizeof(int));

        add<<<1,1>>>(a, b, dev_c);

        cudaMemcpy(&c, dev_c, sizeof(int), cudaMemcpyDeviceToHost);
        cudaFree(dev_c);

        cout << a << "+" << b << "=" << c <<"." << endl;

        return 0;
}
```

**程序分析**：
理由GPU实现两个整数的相加。

- 在调用函数GPU设备上运行的函数前需要先调用`cudaMalloc`在GPU上开辟相应的内存，用于保存运行结果和参数；
- 函数调用完成后需要调用`cudaFree`函数来释放在GPU设备上开辟的内存；
- 主机和GPU设备之间的内存拷贝使用`cudaMemcpy`函数，和普通的C语言中的`memcpy`调用方式一样，只是多了一个参数，指定拷贝的方向：
    - `cudaMemcpyDeviceToHost`:从GPU设备内存拷贝的CPU主机内存；
    - `cudaMemcpyHostToDevice`:从CPU主机内存拷贝到GPU设备内存。